In [1]:
import json

PATH = json.load(open('path_config.json'))

In [2]:
test_source_path = PATH['xsum_fariseq'] + '/test.source'
test_target_path = PATH['xsum_fariseq'] + '/test.target'
xsum_test_prediction = 'preds/xsum_official.hypo'

In [3]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

In [4]:
xsum_source = read_lines(test_source_path)
xsum_target = read_lines(test_target_path)
xsum_preds = read_lines(xsum_test_prediction)
assert len(xsum_source) == len(xsum_target) == len(xsum_preds)
print(len(xsum_source))

11301


#### Annotation

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [6]:
# import nltk

# from nltk.tokenize import word_tokenize
# from nltk.tag import pos_tag

In [7]:
ex = 'Middlesbrough midfielder Federico Fazio has left the Championship club after his contract was terminated by mutual consent.'

In [8]:
# ne_tree = nltk.ne_chunk(pos_tag(word_tokenize(xsum_preds[ID])))
# # print(ne_tree)

#### Build Dataset

In [9]:
import json
import copy

from random import randint

In [10]:
# format:
# {
#     'id': 0,
#     'pred': 'This is a test.',
#     'ents': [{'start': 62,official_predsnd': 75, 'type': 'CARDINAL', 'ent': 'European', 'label': 0},
#              {'start': 91, 'end': 93, 'type': 'CARDINAL', 'ent': 'European', 'label': 0},
#              {'start': 109, 'end': 127, 'type': 'CARDINAL', 'ent': 'European', 'label': 0}
#             ],
#     'hallucinations': ['more than 100 jobs']
# }
# LABEL: 
# 0 non-hallucination
# 1 true hallucination
# 2 false hallucination

annotated = [
{'id': 8805,
 'pred': 'A biscuit maker has gone into administration with the loss of more than 100 jobs after the UK voted to leave the European Union.',
 'ents': [{'start': 62, 'end': 75, 'label': 2, 'type': 'CARDINAL', 'ent': 'more than 100'},
          {'start': 91, 'end': 93, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 109, 'end': 127, 'label': 0, 'type': 'ORG', 'ent': 'the European Union'}],
 'hallucinations': ['more than 100']
},
{'id': 9444,
 'pred': 'Twin-to-twin transfusion syndrome (TTTS) is being tracked by a hospital in Cardiff in a bid to save the lives of babies born with the condition.',
 'ents': [{'start': 35, 'end': 39, 'label': 0, 'type': 'ORG', 'ent': 'TTTS'},
          {'start': 75, 'end': 82, 'label': 2, 'type': 'LOC', 'ent': 'Cardiff'}],
 'hallucinations': ['in Cardiff']
},
{'id': 1431,
 'pred': 'Middlesbrough midfielder Federico Fazio has left the Premier club after his contract was terminated by mutual consent.',
 'ents': [{'start': 0, 'end': 13, 'label': 1, 'type': 'ORG', 'ent': 'Middlesbrough'},
          {'start': 25, 'end': 33, 'label': 2, 'type': 'PERSON', 'ent': 'Federico'},
          {'start': 34, 'end': 39, 'label': 2, 'type': 'PERSON', 'ent': 'Fazio'}],
 'hallucinations': ['Middlesbrough midfielder Federico Fazio']
},
{'id': 7079,
 'pred': "New Celtic manager Brendan Rodgers has met the club's captain for the first time as he prepares for his first game in charge of the club.",
 'ents': [{'start': 0, 'end': 10, 'label': 1, 'type': 'ORG', 'ent': 'New Celtic'},
          {'start': 19, 'end': 26, 'label': 1, 'type': 'PERSON', 'ent': 'Brendan'},
          {'start': 27, 'end': 34, 'label': 0, 'type': 'PERSON', 'ent': 'Rodgers'},
          {'start': 70, 'end': 75, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 104, 'end': 109, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'}],
 'hallucinations': ['New Celtic', 'Brendan']
},
{'id': 2427,
 'pred': 'The Catholic Church should apologise to the families of unwed mothers who died at mother-and-baby homes, the Archbishop of Dublin, Rowan Martin, has said.',
 'ents': [{'start': 0, 'end': 19, 'label': 0, 'type': 'ORG', 'ent': 'The Catholic Church'},
          {'start': 123, 'end': 129, 'label': 2, 'type': 'GPE', 'ent': 'Dublin'},
          {'start': 131, 'end': 136, 'label': 2, 'type': 'PERSON', 'ent': 'Rowan'},
          {'start': 137, 'end': 143, 'label': 0, 'type': 'PERSON', 'ent': 'Martin'}],
 'hallucinations': ['Dublin', 'Rowan']
},
{'id': 6220,
 'pred': 'The body of a man whose body was found at the site of the Swansea Bay Power Station collapse has been removed from the site.',
 'ents': [{'start': 58, 'end': 69, 'label': 3, 'type': 'GPE', 'ent': 'Swansea Bay'},
          {'start': 70, 'end': 83, 'label': 1, 'type': 'ORG', 'ent': 'Power Station'}],
 'hallucinations': ['Swansea Bay Power Station']
},
{'id': 5860,
 'pred': 'The government has announced a voluntary higher rate of the London living wage, which will rise to £9.40 an hour from 1 July.',
 'ents': [{'start': 60, 'end': 66, 'label': 0, 'type': 'GPE', 'ent': 'London'},
          {'start': 100, 'end': 104, 'label': 0, 'type': 'MONEY', 'ent': '9.40'},
          {'start': 118, 'end': 124, 'label': 2, 'type': 'DATE', 'ent': '1 July'}],
 'hallucinations': ['from 1 July']
},
{'id': 10579,
 'pred': 'Sydney has marked the first anniversary of the siege at the cafe where two women were killed by a gunman in the city last year.',
 'ents': [{'start': 0, 'end': 6, 'label': 0, 'type': 'GPE', 'ent': 'Sydney'},
          {'start': 22, 'end': 27, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 71, 'end': 74, 'label': 0, 'type': 'CARDINAL', 'ent': 'two'},
          {'start': 117, 'end': 126, 'label': 0, 'type': 'DATE', 'ent': 'last year'}],
 'hallucinations': ['two women']
},
{'id': 5455,
 'pred': 'As Abid Naseer went on trial in the United States for his role in an al-Qaeda terror plot to carry out a "mass-casualty" bomb attack in Manchester, BBC News looks at what police believe would have happened if he had not been caught.',
 'ents': [{'start': 3, 'end': 7, 'label': 0, 'type': 'PERSON', 'ent': 'Abid'},
          {'start': 8, 'end': 14, 'label': 0, 'type': 'PERSON', 'ent': 'Naseer'},
          {'start': 32, 'end': 49, 'label': 0, 'type': 'GPE', 'ent': 'the United States'},
          {'start': 69, 'end': 77, 'label': 0, 'type': 'ORG', 'ent': 'al-Qaeda'},
          {'start': 136, 'end': 146, 'label': 0, 'type': 'GPE', 'ent': 'Manchester'},
          {'start': 148, 'end': 156, 'label': 1, 'type': 'ORG', 'ent': 'BBC News'}],
 'hallucinations': []
},
{'id': 2407,
 'pred': "The world's biggest asset manager, BlackRock, is planning to cut up to 1,000 jobs as part of a cost-cutting drive, according to reports.",
 'ents': [{'start': 35, 'end': 44, 'label': 0, 'type': 'ORG', 'ent': 'BlackRock'},
          {'start': 65, 'end': 76, 'label': 2, 'type': 'CARDINAL', 'ent': 'up to 1,000'}],
 'hallucinations': ['up to 1,000 jobs']
},
{'id': 416,
 'pred': 'Crawley Town have signed Dutch striker Marvin Verheydt from Dutch club Maastricht for an undisclosed fee, subject to international clearance.',
 'ents': [{'start': 0, 'end': 12, 'label': 1, 'type': 'ORG', 'ent': 'Crawley Town'},
          {'start': 25, 'end': 30, 'label': 1, 'type': 'GPE', 'ent': 'Dutch'},
          {'start': 39, 'end': 45, 'label': 2, 'type': 'PERSON', 'ent': 'Marvin'},
          {'start': 46, 'end': 54, 'label': 0, 'type': 'PERSON', 'ent': 'Verheydt'},
          {'start': 60, 'end': 65, 'label': 1, 'type': 'GPE', 'ent': 'Dutch'},
          {'start': 71, 'end': 81, 'label': 0, 'type': 'ORG', 'ent': 'Maastricht'}],
 'hallucinations': []
},
{'id': 6824,
 'pred': 'Leigh Centurions have agreed a deal to sign Salford Red Devils half-back Ryan Chase on a free transfer after he was cleared to leave the club following a disciplinary hearing.',
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'ORG', 'ent': 'Leigh'},
          {'start': 6, 'end': 16, 'label': 1, 'type': 'ORG', 'ent': 'Centurions'},
          {'start': 44, 'end': 51, 'label': 0, 'type': 'GPE', 'ent': 'Salford'},
          {'start': 52, 'end': 62, 'label': 0, 'type': 'ORG', 'ent': 'Red Devils'},
          {'start': 73, 'end': 77, 'label': 2, 'type': 'PERSON', 'ent': 'Ryan'},
          {'start': 78, 'end': 83, 'label': 0, 'type': 'PERSON', 'ent': 'Chase'}],
 'hallucinations': ['Centurions', 'Ryan']
},
{'id': 1513,
 'pred': 'Manchester United have failed to complete the signing of Spain goalkeeper David de Gea from Manchester United to Real Madrid in a deal that was due to be completed on Monday night, but the transfer window is still open in Spain.',
 'ents': [{'start': 0, 'end': 17, 'label': 1, 'type': 'ORG', 'ent': 'Manchester United'},
          {'start': 57, 'end': 62, 'label': 0, 'type': 'GPE', 'ent': 'Spain'},
          {'start': 74, 'end': 79, 'label': 1, 'type': 'PERSON', 'ent': 'David'},
          {'start': 80, 'end': 82, 'label': 0, 'type': 'PERSON', 'ent': 'de'},
          {'start': 83, 'end': 86, 'label': 0, 'type': 'PERSON', 'ent': 'Gea'},
          {'start': 92, 'end': 109, 'label': 1, 'type': 'ORG', 'ent': 'Manchester United'},
          {'start': 113, 'end': 124, 'label': 0, 'type': 'ORG', 'ent': 'Real Madrid'},
          {'start': 167, 'end': 179, 'label': 2, 'type': 'TIME', 'ent': 'Monday night'},
          {'start': 222, 'end': 227, 'label': 0, 'type': 'GPE', 'ent': 'Spain'}],
 'hallucinations': []
},
{'id': 1819,
 'pred': 'The brother of one of the victims of Bloody Sunday has said plans to hold a march by former soldiers in Londonderry are "totally insensitive" to the people of the city.',
 'ents': [{'start': 15, 'end': 18, 'label': 0, 'type': 'CARDINAL', 'ent': 'one'},
          {'start': 37, 'end': 50, 'label': 0, 'type': 'EVENT', 'ent': 'Bloody Sunday'},
          {'start': 104, 'end': 115, 'label': 0, 'type': 'GPE', 'ent': 'Londonderry'}],
 'hallucinations': []
},
{'id': 560,
 'pred': 'South Africa will appoint a new head coach ahead of their tour of England next month after the end of the current one-day international and Test series against Sri Lanka, reports BBC Sport.',
 'ents': [{'start': 0, 'end': 12, 'label': 0, 'type': 'GPE', 'ent': 'South Africa'},
          {'start': 66, 'end': 73, 'label': 0, 'type': 'GPE', 'ent': 'England'},
          {'start': 74, 'end': 84, 'label': 1, 'type': 'DATE', 'ent': 'next month'},
          {'start': 114, 'end': 121, 'label': 0, 'type': 'DATE', 'ent': 'one-day'},
          {'start': 160, 'end': 169, 'label': 0, 'type': 'GPE', 'ent': 'Sri Lanka'},
          {'start': 179, 'end': 188, 'label': 1, 'type': 'ORG', 'ent': 'BBC Sport'}],
 'hallucinations': ['next month', 'reports BBC Sport']
},
{'id': 4174,
 'pred': 'A British tourist who was jailed in Malaysia for indecency after stripping off at a mountain peak has left the country on a flight to Kuala Lumpur.',
 'ents': [{'start': 2, 'end': 9, 'label': 0, 'type': 'NORP', 'ent': 'British'},
          {'start': 36, 'end': 44, 'label': 0, 'type': 'GPE', 'ent': 'Malaysia'},
          {'start': 134, 'end': 146, 'label': 0, 'type': 'GPE', 'ent': 'Kuala Lumpur'}],
 'hallucinations': []
},
{'id': 2194,
 'pred': '"I\'m a very good storyteller," says David Chase, as he tells the story of his life.',
 'ents': [{'start': 36, 'end': 41, 'label': 2, 'type': 'PERSON', 'ent': 'David'},
          {'start': 42, 'end': 47, 'label': 0, 'type': 'PERSON', 'ent': 'Chase'}],
 'hallucinations': []
},
{'id': 2517,
 'pred': "Saturday's League One game between Oldham Athletic and Bolton Wanderers has been postponed because of a waterlogged pitch.",
 'ents': [{'start': 0, 'end': 8, 'label': 1, 'type': 'DATE', 'ent': 'Saturday'},
          {'start': 18, 'end': 21, 'label': 1, 'type': 'CARDINAL', 'ent': 'One'},
          {'start': 35, 'end': 50, 'label': 1, 'type': 'ORG', 'ent': 'Oldham Athletic'},
          {'start': 55, 'end': 71, 'label': 2, 'type': 'ORG', 'ent': 'Bolton Wanderers'}],
 'hallucinations': ["Saturday's League One", 'Oldham Athletic', 'Bolton Wanderers']
},
{'id': 922,
 'pred': 'The way GCSEs are graded in England has failed to meet five conditions set out by the exams watchdog Ofqual, according to a new report.',
 'ents': [{'start': 28, 'end': 35, 'label': 0, 'type': 'GPE', 'ent': 'England'},
          {'start': 55, 'end': 59, 'label': 0, 'type': 'CARDINAL', 'ent': 'five'},
          {'start': 101, 'end': 107, 'label': 0, 'type': 'ORG', 'ent': 'Ofqual'}],
 'hallucinations': []
},
{'id': 10066,
 'pred': 'Police investigating the disappearance of a man who went missing in May have launched a national appeal for information about his disappearance.',
 'ents': [{'start': 68, 'end': 71, 'label': 0, 'type': 'DATE', 'ent': 'May'}],
 'hallucinations': []
},
{'id': 1367,
 'pred': 'Jockey David Mullins rode Rule The World to victory in the Grand National at Aintree for the first time at the age of 19.',
 'ents': [{'start': 0, 'end': 6, 'label': 0, 'type': 'PERSON', 'ent': 'Jockey'},
          {'start': 7, 'end': 12, 'label': 0, 'type': 'PERSON', 'ent': 'David'},
          {'start': 13, 'end': 20, 'label': 0, 'type': 'PERSON', 'ent': 'Mullins'},
          {'start': 55, 'end': 84, 'label': 0, 'type': 'ORG', 'ent': 'the Grand National at Aintree'},
          {'start': 93, 'end': 98, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 107, 'end': 120, 'label': 0, 'type': 'DATE', 'ent': 'the age of 19'}],
 'hallucinations': []
},
{'id': 11207,
 'pred': 'The jury in the trial of the man accused of carrying out the Boston Marathon bombings has heard graphic testimony about the injuries suffered by the victims of the attack.',
 'ents': [{'start': 61, 'end': 76, 'label': 1, 'type': 'ORG', 'ent': 'Boston Marathon'}],
 'hallucinations': []
},
{'id': 3211,
 'pred': 'A man has been charged in the US state of Michigan after a student with a severe peanut allergy was allegedly forced to eat peanut butter as part of a fraternity initiation ritual.',
 'ents': [{'start': 30, 'end': 32, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 42, 'end': 50, 'label': 0, 'type': 'GPE', 'ent': 'Michigan'}],
 'hallucinations': []
},
{'id': 10524,
 'pred': "Edinburgh Zoo's giant panda Tian Tian is expected to give birth at the end of the month, the zoo has said.",
 'ents': [{'start': 0, 'end': 15, 'label': 0, 'type': 'ORG', 'ent': "Edinburgh Zoo's"},
          {'start': 28, 'end': 32, 'label': 0, 'type': 'PERSON', 'ent': 'Tian'},
          {'start': 33, 'end': 37, 'label': 0, 'type': 'PERSON', 'ent': 'Tian'},
          {'start': 67, 'end': 87, 'label': 0, 'type': 'DATE', 'ent': 'the end of the month'}],
 'hallucinations': []
},
{'id': 10488,
 'pred': 'Pupils who are able and talented are not being given enough support in the education system, according to the chief inspector of schools at Estyn.',
 'ents': [{'start': 140, 'end': 145, 'label': 0, 'type': 'LOC', 'ent': 'Estyn'}],
 'hallucinations': []
},
{'id': 4809,
 'pred': 'US Secretary of State Rex Tillerson has said he was "convinced" by his wife to take up the job by President Donald Trump, who asked him to be his secretary of state.',
 'ents': [{'start': 0, 'end': 2, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 22, 'end': 25, 'label': 0, 'type': 'PERSON', 'ent': 'Rex'},
          {'start': 26, 'end': 35, 'label': 0, 'type': 'PERSON', 'ent': 'Tillerson'},
          {'start': 108, 'end': 114, 'label': 0, 'type': 'PERSON', 'ent': 'Donald'},
          {'start': 115, 'end': 120, 'label': 0, 'type': 'PERSON', 'ent': 'Trump'}],
 'hallucinations': []
},
{'id': 6228,
 'pred': 'A group of MPs has called on the government to legalise medical cannabis after a study found that one million people across the UK rely on the drug for medical reasons, but there is limited or no convincing evidence that it works.',
 'ents': [{'start': 98, 'end': 109, 'label': 0, 'type': 'CARDINAL', 'ent': 'one million'},
          {'start': 128, 'end': 130, 'label': 0, 'type': 'GPE', 'ent': 'UK'}],
 'hallucinations': []
},
{'id': 10517,
 'pred': 'Police ombudsman Michael Maguire has said he welcomes the re-opening of historical investigations into allegations of wrongdoing by the PSNI in Northern Ireland.',
 'ents': [{'start': 17, 'end': 24, 'label': 0, 'type': 'PERSON', 'ent': 'Michael'},
          {'start': 25, 'end': 32, 'label': 0, 'type': 'PERSON', 'ent': 'Maguire'},
          {'start': 136, 'end': 140, 'label': 0, 'type': 'ORG', 'ent': 'PSNI'},
          {'start': 144, 'end': 160, 'label': 0, 'type': 'GPE', 'ent': 'Northern Ireland'}],
 'hallucinations': []
},
{'id': 7247,
 'pred': 'Dundee Stars head coach Marc LeFebvre hopes his side can learn from their experience at the Elite League play-off finals and return to the finals next season.',
 'ents': [{'start': 0, 'end': 12, 'label': 0, 'type': 'ORG', 'ent': 'Dundee Stars'},
          {'start': 24, 'end': 28, 'label': 0, 'type': 'PERSON', 'ent': 'Marc'},
          {'start': 29, 'end': 37, 'label': 0, 'type': 'PERSON', 'ent': 'LeFebvre'},
          {'start': 92, 'end': 104, 'label': 0, 'type': 'ORG', 'ent': 'Elite League'},
          {'start': 146, 'end': 157, 'label': 0, 'type': 'DATE', 'ent': 'next season'}],
 'hallucinations': []
},
{'id': 2724,
 'pred': 'Cardiff and its city region should be given more power by the Welsh assembly to work better with the "Northern Powerhouse" region of England, the chairman of the Northern Powerhouse group has said.',
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'ORG', 'ent': 'Cardiff'},
          {'start': 62, 'end': 67, 'label': 0, 'type': 'GPE', 'ent': 'Welsh'},
          {'start': 133, 'end': 140, 'label': 0, 'type': 'GPE', 'ent': 'England'},
          {'start': 162, 'end': 181, 'label': 0, 'type': 'ORG', 'ent': 'Northern Powerhouse'}],
 'hallucinations': []
},
{'id': 4630,
 'pred': 'Paralympic gold medallist Bethany Brown says she will not change her bow to recurve in order to qualify for the 2016 Rio Games.',
 'ents': [{'start': 26, 'end': 33, 'label': 2, 'type': 'PERSON', 'ent': 'Bethany'},
          {'start': 34, 'end': 39, 'label': 0, 'type': 'PERSON', 'ent': 'Brown'},
          {'start': 112, 'end': 116, 'label': 0, 'type': 'DATE', 'ent': '2016'}],
 'hallucinations': ['Bethany']
},
{'id': 6126,
 'pred': 'The Supreme Court has ruled that the assembly has the power to set minimum wages for agricultural workers in Wales.',
 'ents': [{'start': 0, 'end': 17, 'label': 0, 'type': 'ORG', 'ent': 'The Supreme Court'},
          {'start': 109, 'end': 114, 'label': 0, 'type': 'GPE', 'ent': 'Wales'}],
 'hallucinations': []
},
{'id': 3529,
 'pred': 'The daughter of the first baby to be born on an air ambulance has joined Loganair as a cabin crew attendant, 40 years after her birth.',
 'ents': [{'start': 20, 'end': 25, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 73, 'end': 81, 'label': 0, 'type': 'PERSON', 'ent': 'Loganair'},
          {'start': 109, 'end': 117, 'label': 0, 'type': 'DATE', 'ent': '40 years'}],
 'hallucinations': []
},
{'id': 11006,
 'pred': 'Bristol City moved three points clear at the top of League One with a hard-fought victory over Ipswich Town.',
 'ents': [{'start': 0, 'end': 12, 'label': 0, 'type': 'GPE', 'ent': 'Bristol City'},
          {'start': 19, 'end': 24, 'label': 3, 'type': 'CARDINAL', 'ent': 'three'},
          {'start': 52, 'end': 62, 'label': 0, 'type': 'ORG', 'ent': 'League One'},
          {'start': 95, 'end': 107, 'label': 0, 'type': 'FAC', 'ent': 'Ipswich Town'}],
 'hallucinations': ['moved three points clear']
},
{'id': 8447,
 'pred': 'The White House has confirmed that President Barack Obama and First Lady Michelle Obama were among the celebrities who attended a private party in New York to celebrate their wedding anniversary.',
 'ents': [{'start': 0, 'end': 15, 'label': 1, 'type': 'ORG', 'ent': 'The White House'},
          {'start': 45, 'end': 51, 'label': 1, 'type': 'PERSON', 'ent': 'Barack'},
          {'start': 52, 'end': 57, 'label': 0, 'type': 'PERSON', 'ent': 'Obama'},
          {'start': 62, 'end': 67, 'label': 1, 'type': 'ORDINAL', 'ent': 'First'},
          {'start': 73, 'end': 81, 'label': 1, 'type': 'PERSON', 'ent': 'Michelle'},
          {'start': 82, 'end': 87, 'label': 0, 'type': 'PERSON', 'ent': 'Obama'},
          {'start': 147, 'end': 155, 'label': 3, 'type': 'GPE', 'ent': 'New York'}],
 'hallucinations': ['The White House', 'Barack', 'First Lady Michelle', 'to celebrate their wedding anniversary']
},
{'id': 9980,
 'pred': 'Tottenham Hotspur have agreed to pay £1m to the family of a 17-year-old boy who suffered brain injuries after signing for the club despite having a potentially fatal heart condition.',
 'ents': [{'start': 0, 'end': 9, 'label': 0, 'type': 'GPE', 'ent': 'Tottenham'},
          {'start': 10, 'end': 17, 'label': 0, 'type': 'GPE', 'ent': 'Hotspur'},
          {'start': 37, 'end': 40, 'label': 2, 'type': 'MONEY', 'ent': '£1m'},
          {'start': 60, 'end': 71, 'label': 3, 'type': 'DATE', 'ent': '17-year-old'}],
 'hallucinations': ['£1m', '17-year-old']
},
{'id': 8685,
 'pred': 'League One side Wigan Athletic have signed Bournemouth midfielder Chris MacDonald on loan until the end of the season.',
 'ents': [{'start': 7, 'end': 10, 'label': 0, 'type': 'CARDINAL', 'ent': 'One'},
          {'start': 16, 'end': 30, 'label': 1, 'type': 'ORG', 'ent': 'Wigan Athletic'},
          {'start': 43, 'end': 54, 'label': 1, 'type': 'LOC', 'ent': 'Bournemouth'},
          {'start': 66, 'end': 71, 'label': 2, 'type': 'PERSON', 'ent': 'Chris'},
          {'start': 72, 'end': 81, 'label': 0, 'type': 'PERSON', 'ent': 'MacDonald'},
          {'start': 96, 'end': 117, 'label': 2, 'type': 'DATE', 'ent': 'the end of the season'}],
 'hallucinations': ['Wigan Athletic', 'Bournemouth midfielder', 'Chris', 'on loan until the end of the season']
},
{'id': 11080,
 'pred': 'The new head of the Public Services Complaints Commission for Wales has been appointed.',
 'ents': [{'start': 16, 'end': 57, 'label': 2, 'type': 'ORG', 'ent': 'the Public Services Complaints Commission'},
          {'start': 62, 'end': 67, 'label': 0, 'type': 'GPE', 'ent': 'Wales'}],
 'hallucinations': ['the Public Services Complaints Commission']
},
{'id': 8558,
 'pred': 'A Conservative MP has said he is "personally very disappointed" that plans to devolve tax-raising powers to the Welsh Government without a referendum have been rejected by the Welsh Secretary.',
 'ents': [{'start': 2, 'end': 17, 'label': 0, 'type': 'ORG', 'ent': 'Conservative MP'},
          {'start': 108, 'end': 128, 'label': 0, 'type': 'ORG', 'ent': 'the Welsh Government'},
          {'start': 176, 'end': 181, 'label': 0, 'type': 'NORP', 'ent': 'Welsh'}],
 'hallucinations': []
},
{'id': 6931,
 'pred': "Prime Minister Narendra Modi's visit to Silicon Valley is the first by an Indian prime minister to the US in more than a decade, and the first since he took office in May 2014.",
 'ents': [{'start': 15,
   'end': 23,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Narendra'},
  {'start': 24, 'end': 28, 'label': 0, 'type': 'PERSON', 'ent': 'Modi'},
  {'start': 40,
   'end': 54,
   'label': 0,
   'type': 'LOC',
   'ent': 'Silicon Valley'},
  {'start': 74, 'end': 80, 'label': 0, 'type': 'NORP', 'ent': 'Indian'},
  {'start': 103, 'end': 105, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 109,
   'end': 127,
   'label': 0,
   'type': 'DATE',
   'ent': 'more than a decade'},
  {'start': 137, 'end': 142, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
  {'start': 167, 'end': 175, 'label': 0, 'type': 'DATE', 'ent': 'May 2014'}],
 'hallucinations': []
},
{'id': 6373,
 'pred': 'Police investigating the disappearance of an RAF serviceman who went missing after a night out in Bury St Edmunds have released CCTV images of three teenagers they want to trace.',
 'ents': [{'start': 45, 'end': 48, 'label': 0, 'type': 'ORG', 'ent': 'RAF'},
          {'start': 83, 'end': 90, 'label': 0, 'type': 'TIME', 'ent': 'a night'},
          {'start': 98, 'end': 113, 'label': 0, 'type': 'GPE', 'ent': 'Bury St Edmunds'},
          {'start': 128, 'end': 132, 'label': 0, 'type': 'ORG', 'ent': 'CCTV'},
          {'start': 143, 'end': 148, 'label': 0, 'type': 'CARDINAL', 'ent': 'three'}],
 'hallucinations': []
},
{'id': 9961,
 'pred': 'Three Syrian refugees who overpowered a suspected terror suspect in the German city of Leipzig and tied him up have been hailed as "heroes" by the country\'s media and politicians.',
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'CARDINAL', 'ent': 'Three'},
          {'start': 6, 'end': 12, 'label': 0, 'type': 'NORP', 'ent': 'Syrian'},
          {'start': 72, 'end': 78, 'label': 0, 'type': 'NORP', 'ent': 'German'},
          {'start': 87, 'end': 94, 'label': 0, 'type': 'GPE', 'ent': 'Leipzig'}],
 'hallucinations': []
},
{'id': 6235,
 'pred': "China's Tiangong-1 space station will burn up when it re-enters the Earth's atmosphere later this week, according to the country's space agency.",
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'GPE', 'ent': 'China'},
          {'start': 8, 'end': 18, 'label': 0, 'type': 'FAC', 'ent': 'Tiangong-1'},
          {'start': 87, 'end': 102, 'label': 2, 'type': 'DATE', 'ent': 'later this week'}],
 'hallucinations': []
},
{'id': 5414,
 'pred': 'Chinese search giant Baidu has unveiled a voice-activated artificial intelligence (AI) that can be used on a smart speaker.',
 'ents': [{'start': 0,
   'end': 7,
   'label': 0,
   'type': 'NORP',
   'ent': 'Chinese'},
  {'start': 21, 'end': 26, 'label': 0, 'type': 'ORG', 'ent': 'Baidu'}],
 'hallucinations': []
},
{'id': 1168,
 'pred': 'Belfast-based Mexican restaurant chain Boojum has been sold for an undisclosed sum to a group of investors in the Republic of Ireland.',
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'GPE', 'ent': 'Belfast'},
          {'start': 14, 'end': 21, 'label': 0, 'type': 'NORP', 'ent': 'Mexican'},
          {'start': 39, 'end': 45, 'label': 0, 'type': 'PERSON', 'ent': 'Boojum'},
          {'start': 110, 'end': 133, 'label': 1, 'type': 'GPE', 'ent': 'the Republic of Ireland'}],
 'hallucinations': []
},
{'id': 7987,
 'pred': "The Northern Ireland Chamber of Commerce has said that the renegotiation of the UK's relationship with the European Union (EU) is having little impact on day-to-day business in Northern Ireland.",
 'ents': [{'start': 4, 'end': 20, 'label': 0, 'type': 'NORP', 'ent': 'Northern Ireland'},
          {'start': 21, 'end': 40, 'label': 0, 'type': 'ORG', 'ent': 'Chamber of Commerce'},
          {'start': 80, 'end': 82, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 103, 'end': 121, 'label': 0, 'type': 'ORG', 'ent': 'the European Union'},
          {'start': 123, 'end': 125, 'label': 0, 'type': 'ORG', 'ent': 'EU'},
          {'start': 177, 'end': 193, 'label': 0, 'type': 'GPE', 'ent': 'Northern Ireland'}],
 'hallucinations': []
},
{'id': 9128,
 'pred': 'More than a quarter of teachers in Wales have been banned from the profession for misconduct involving sexual content on social media in the past three years.',
 'ents': [{'start': 0, 'end': 19, 'label': 2, 'type': 'CARDINAL', 'ent': 'More than a quarter'},
          {'start': 35, 'end': 40, 'label': 0, 'type': 'GPE', 'ent': 'Wales'},
          {'start': 137, 'end': 157, 'label': 2, 'type': 'DATE', 'ent': 'the past three years'}],
 'hallucinations': ['More than a quarter', 'the past three years']
},
{'id': 8495,
 'pred': 'British pair Naomi Broady and Heather Watson were beaten 6-3 6-2 by Chan Yung-jan and Chan Shuai-Wei in the doubles final of the Hong Kong Open.',
 'ents': [{'start': 0, 'end': 7, 'label': 1, 'type': 'NORP', 'ent': 'British'},
          {'start': 13, 'end': 18, 'label': 1, 'type': 'PERSON', 'ent': 'Naomi'},
          {'start': 19, 'end': 25, 'label': 0, 'type': 'PERSON', 'ent': 'Broady'},
          {'start': 30, 'end': 37, 'label': 1, 'type': 'PERSON', 'ent': 'Heather'},
          {'start': 38, 'end': 44, 'label': 0, 'type': 'PERSON', 'ent': 'Watson'},
          {'start': 57, 'end': 60, 'label': 1, 'type': 'CARDINAL', 'ent': '6-3'},
          {'start': 61, 'end': 64, 'label': 2, 'type': 'CARDINAL', 'ent': '6-2'},
          {'start': 68, 'end': 72, 'label': 0, 'type': 'PERSON', 'ent': 'Chan'},
          {'start': 73, 'end': 81, 'label': 1, 'type': 'PERSON', 'ent': 'Yung-jan'},
          {'start': 86, 'end': 90, 'label': 0, 'type': 'PERSON', 'ent': 'Chan'},
          {'start': 91, 'end': 100, 'label': 2, 'type': 'PERSON', 'ent': 'Shuai-Wei'},
          {'start': 125, 'end': 143, 'label': 1, 'type': 'ORG', 'ent': 'the Hong Kong Open'}],
 'hallucinations': ['British', 'Naomi', 'Heather', '6-3 6-2', 'Yung-jan', 'Shuai-Wei', 'the Hong Kong Open']
},
{'id': 6486,
 'pred': 'North Korean leader Kim Jong-un has threatened to launch a "fire and fury" missile attack on the US island of Guam, in what has been described as a warning to President Donald Trump.',
 'ents': [{'start': 0, 'end': 12, 'label': 0, 'type': 'NORP', 'ent': 'North Korean'},
          {'start': 20, 'end': 23, 'label': 0, 'type': 'PERSON', 'ent': 'Kim'},
          {'start': 24, 'end': 31, 'label': 0, 'type': 'PERSON', 'ent': 'Jong-un'},
          {'start': 97, 'end': 99, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 110, 'end': 114, 'label': 0, 'type': 'GPE', 'ent': 'Guam'},
          {'start': 169, 'end': 175, 'label': 0, 'type': 'PERSON', 'ent': 'Donald'},
          {'start': 176, 'end': 181, 'label': 0, 'type': 'PERSON', 'ent': 'Trump'}],
 'hallucinations': []
},
{'id': 1584,
 'pred': 'Northampton Town have been placed into administration by Northampton Borough Council.',
 'ents': [{'start': 0, 'end': 11, 'label': 0, 'type': 'GPE', 'ent': 'Northampton'},
          {'start': 57, 'end': 84, 'label': 0, 'type': 'ORG', 'ent': 'Northampton Borough Council'}],
 'hallucinations': []
},
{'id': 10958,
 'pred': 'Police have said the death of a man at a hotel in Edinburgh is not being treated as suspicious after his body was found.',
 'ents': [{'start': 50,
   'end': 59,
   'label': 1,
   'type': 'GPE',
   'ent': 'Edinburgh'}],
 'hallucinations': ['in Edinburgh']
},
{'id': 11017,
 'pred': 'The RSPCA has called on Wrexham council to drop plans to ban dogs from play areas and sports pitches, saying it could punish responsible dog owners.',
 'ents': [{'start': 4, 'end': 9, 'label': 0, 'type': 'ORG', 'ent': 'RSPCA'},
          {'start': 24, 'end': 31, 'label': 0, 'type': 'GPE', 'ent': 'Wrexham'}],
 'hallucinations': []
},
{'id': 11037,
 'pred': 'A buzzard has been released back into the wild after becoming stuck in a fence during a chase in Cheshire.',
 'ents': [{'start': 97, 'end': 105, 'label': 2, 'type': 'GPE', 'ent': 'Cheshire'}],
 'hallucinations': ['in Cheshire']
},
{'id': 5148,
 'pred': 'Five children at a Neath Port Talbot nursery have been diagnosed with measles, health officials have said.',
 'ents': [{'start': 0, 'end': 4, 'label': 3, 'type': 'CARDINAL', 'ent': 'Five'},
          {'start': 19, 'end': 24, 'label': 0, 'type': 'LOC', 'ent': 'Neath'},
          {'start': 25, 'end': 36, 'label': 1, 'type': 'GPE', 'ent': 'Port Talbot'}],
 'hallucinations': ['Port Talbot']
},

{'id': 10380,
 'pred': 'A Palestinian woman who was seriously injured in a firebombing in the occupied West Bank that killed her toddler son and her husband has died in hospital, her family says.',
 'ents': [{'start': 2, 'end': 13, 'label': 0, 'type': 'NORP', 'ent': 'Palestinian'},
          {'start': 79, 'end': 88, 'label': 0, 'type': 'GPE', 'ent': 'West Bank'}],
 'hallucinations': ['in a firebombing', 'her family says']
},
{'id': 5747,
 'pred': 'Six teenagers have died from carbon monoxide poisoning after a party at a remote hut in northern Germany, police say.',
 'ents': [{'start': 0, 'end': 3, 'label': 0, 'type': 'CARDINAL', 'ent': 'Six'},
          {'start': 97, 'end': 104, 'label': 0, 'type': 'GPE', 'ent': 'Germany'}],
 'hallucinations': []
},
{'id': 10943,
 'pred': "A powerful cyclone has killed at least 11 people and injured more than 100 in Vanuatu, the Pacific nation's president has said.",
 'ents': [{'start': 30, 'end': 41, 'label': 2, 'type': 'CARDINAL', 'ent': 'at least 11'},
          {'start': 61, 'end': 74, 'label': 2, 'type': 'CARDINAL', 'ent': 'more than 100'},
          {'start': 78, 'end': 85, 'label': 0, 'type': 'GPE', 'ent': 'Vanuatu'},
          {'start': 91, 'end': 98, 'label': 1, 'type': 'LOC', 'ent': 'Pacific'}],
 'hallucinations': ['killed at least 11 people and injured more than 100', "the Pacific nation's president has said."]
},
{'id': 9369,
 'pred': 'A Canadian man has described the moment he was struck by lightning during his son-in-law\'s wedding ceremony - just as he was about to tell him he was "a lucky guy".',
 'ents': [{'start': 2, 'end': 10, 'label': 0, 'type': 'NORP', 'ent': 'Canadian'}],
 'hallucinations': []
},
{'id': 9185,
 'pred': 'A pine marten has given birth to its first baby in Wales in two years, according to a wildlife charity.',
 'ents': [{'start': 37, 'end': 42, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 51, 'end': 56, 'label': 0, 'type': 'GPE', 'ent': 'Wales'},
          {'start': 60, 'end': 69, 'label': 0, 'type': 'DATE', 'ent': 'two years'}],
 'hallucinations': []
},
{'id': 3611,
 'pred': "A Rwandan man who called for the mass murder of Tutsi people during the country's genocide has been sentenced to life in prison by a court in Rwanda.",
 'ents': [{'start': 2, 'end': 9, 'label': 0, 'type': 'ORG', 'ent': 'Rwandan'},
          {'start': 48, 'end': 53, 'label': 0, 'type': 'NORP', 'ent': 'Tutsi'},
          {'start': 142, 'end': 148, 'label': 0, 'type': 'GPE', 'ent': 'Rwanda'}],
 'hallucinations': []
},
{'id': 1754,
 'pred': 'Prince William has said he takes his duties as a monarch "very seriously" but does not take them "completely to heart" as he prepares to take on more responsibility.',
 'ents': [{'start': 0, 'end': 6, 'label': 0, 'type': 'PERSON', 'ent': 'Prince'},
          {'start': 7, 'end': 14, 'label': 0, 'type': 'PERSON', 'ent': 'William'}],
 'hallucinations': []
},
{'id': 10244,
 'pred': 'A man has been arrested in China for allegedly driving a Lamborghini at more than twice the legal speed limit around Beijing, state media report.',
 'ents': [{'start': 27, 'end': 32, 'label': 0, 'type': 'GPE', 'ent': 'China'},
          {'start': 57, 'end': 68, 'label': 3, 'type': 'ORG', 'ent': 'Lamborghini'},
          {'start': 117, 'end': 124, 'label': 0, 'type': 'GPE', 'ent': 'Beijing'}],
 'hallucinations': []
},
{'id': 4850,
 'pred': 'The deaths of two pensioners at their home in Ballycastle, County Antrim, are being treated as unexplained, police have said.',
 'ents': [{'start': 14, 'end': 17, 'label': 0, 'type': 'CARDINAL', 'ent': 'two'},
          {'start': 46, 'end': 57, 'label': 0, 'type': 'GPE', 'ent': 'Ballycastle'},
          {'start': 59, 'end': 72, 'label': 1, 'type': 'GPE', 'ent': 'County Antrim'}],
 'hallucinations': ['County Antrim']
},
{'id': 1575,
 'pred': 'Rules requiring schools to promote "fundamental British values", including the role of law, are "deeply ill-considered" and "political posturing", a teachers\' union has warned.',
 'ents': [{'start': 48, 'end': 55, 'label': 0, 'type': 'NORP', 'ent': 'British'}],
 'hallucinations': []
},
{'id': 5686,
 'pred': 'Politicians on both sides of the EU referendum will have to work hard to ensure that the public is not left disappointed by the outcome of the referendum.',
 'ents': [{'start': 33, 'end': 35, 'label': 0, 'type': 'ORG', 'ent': 'EU'}],
 'hallucinations': []
},
{'id': 9254,
 'pred': 'Two men have been found guilty of murdering a man who was shot dead after leaving a pub in Greater Manchester.',
 'ents': [{'start': 0, 'end': 3, 'label': 3, 'type': 'CARDINAL', 'ent': 'Two'},
          {'start': 91, 'end': 109, 'label': 1, 'type': 'GPE', 'ent': 'Greater Manchester'}],
 'hallucinations': []
},
{'id': 2759,
 'pred': 'A chameleon has been put down by RSPCA Cymru after it was found injured and abandoned in a Cardiff park.',
 'ents': [{'start': 33, 'end': 44, 'label': 0, 'type': 'PERSON', 'ent': 'RSPCA Cymru'},
          {'start': 91, 'end': 98, 'label': 1, 'type': 'ORG', 'ent': 'Cardiff'}],
 'hallucinations': []
},
{'id': 8953,
 'pred': 'The BBC is to create a new Ideas Service to bring the "best from Britain\'s great cultural institutions and thinkers" to everyone, as part of a major overhaul of the corporation.',
 'ents': [{'start': 4, 'end': 7, 'label': 0, 'type': 'ORG', 'ent': 'BBC'},
          {'start': 27, 'end': 40, 'label': 0, 'type': 'ORG', 'ent': 'Ideas Service'},
          {'start': 65, 'end': 72, 'label': 0, 'type': 'GPE', 'ent': 'Britain'}],
 'hallucinations': []
},
{'id': 7619,
 'pred': 'An NHS trust has been ordered to pay £1.6m in damages to the family of a baby girl who was left paralysed after being born with severe brain injuries.',
 'ents': [{'start': 3, 'end': 6, 'label': 0, 'type': 'ORG', 'ent': 'NHS'},
          {'start': 38, 'end': 41, 'label': 0, 'type': 'MONEY', 'ent': '1.6'}],
 'hallucinations': []
},
{'id': 700,
 'pred': 'The war on drugs in the United States is at a draw, with more than 600,000 people in jail, $40bn of spending a year in prison, and the number of people caught up in the drug trade increasing by the day.',
 'ents': [{'start': 20, 'end': 37, 'label': 0, 'type': 'GPE', 'ent': 'the United States'},
          {'start': 57, 'end': 74, 'label': 0, 'type': 'CARDINAL', 'ent': 'more than 600,000'},
          {'start': 92, 'end': 96, 'label': 0, 'type': 'MONEY', 'ent': '40bn'}],
 'hallucinations': []
},
{'id': 3379,
 'pred': 'Stagecoach has announced plans to invest more than £100m in new buses and coaches over the next three years.',
 'ents': [{'start': 0, 'end': 10, 'label': 0, 'type': 'ORG', 'ent': 'Stagecoach'},
          {'start': 41, 'end': 56, 'label': 2, 'type': 'CARDINAL', 'ent': 'more than £100m'},
          {'start': 87, 'end': 107, 'label': 2, 'type': 'DATE', 'ent': 'the next three years'}],
 'hallucinations': ['more than £100m', 'over the next three years']
},
{'id': 7021,
 'pred': 'A woman has been given a suspended prison sentence for defrauding a charity set up in memory of a Cumbrian teenager who died from cancer.',
 'ents': [{'start': 98, 'end': 106, 'label': 0, 'type': 'NORP', 'ent': 'Cumbrian'}],
 'hallucinations': []
},
{'id': 3180,
 'pred': 'Burton Albion goalkeeper Connor McLaughlin is one of eight players who will leave the club at the end of the season after being released by the Championship club.',
 'ents': [{'start': 0, 'end': 13, 'label': 0, 'type': 'ORG', 'ent': 'Burton Albion'},
          {'start': 25, 'end': 31, 'label': 2, 'type': 'PERSON', 'ent': 'Connor'},
          {'start': 32, 'end': 42, 'label': 0, 'type': 'PERSON', 'ent': 'McLaughlin'},
          {'start': 46, 'end': 49, 'label': 0, 'type': 'CARDINAL', 'ent': 'one'},
          {'start': 53, 'end': 58, 'label': 2, 'type': 'CARDINAL', 'ent': 'eight'},
          {'start': 94, 'end': 115, 'label': 1, 'type': 'DATE', 'ent': 'the end of the season'}],
 'hallucinations': []
},
{'id': 3030,
 'pred': "Former England and Tottenham midfielder David Ginola has launched his bid to challenge Sepp Blatter for the presidency of world football's governing body Fifa.",
 'ents': [{'start': 7, 'end': 14, 'label': 2, 'type': 'GPE', 'ent': 'England'},
          {'start': 19, 'end': 28, 'label': 0, 'type': 'ORG', 'ent': 'Tottenham'},
          {'start': 40, 'end': 45, 'label': 0, 'type': 'PERSON', 'ent': 'David'},
          {'start': 46, 'end': 52, 'label': 0, 'type': 'PERSON', 'ent': 'Ginola'},
          {'start': 87, 'end': 91, 'label': 1, 'type': 'PERSON', 'ent': 'Sepp'},
          {'start': 92, 'end': 99, 'label': 0, 'type': 'PERSON', 'ent': 'Blatter'},
          {'start': 154, 'end': 158, 'label': 0, 'type': 'ORG', 'ent': 'Fifa'}],
 'hallucinations': []
},
{'id': 191,
 'pred': 'Work to improve a roundabout on the A55 and A483 in Chester has been delayed for a second time, causing "chaos" for motorists.',
 'ents': [{'start': 36, 'end': 39, 'label': 0, 'type': 'ORG', 'ent': 'A55'},
          {'start': 44, 'end': 48, 'label': 0, 'type': 'ORG', 'ent': 'A483'},
          {'start': 52, 'end': 59, 'label': 0, 'type': 'GPE', 'ent': 'Chester'},
          {'start': 83, 'end': 89, 'label': 3, 'type': 'ORDINAL', 'ent': 'second'}],
 'hallucinations': ['for a second time']
},
# {'id': 8770,
#  'pred': 'Everton have signed Oxford United midfielder Luke Kenny on loan until the end of the season, with the option of a permanent move.',
#  'ents': [{'start': 0, 'end': 7, 'label': 2, 'type': 'GPE', 'ent': 'Everton'},
#           {'start': 20, 'end': 33, 'label': 1, 'type': 'ORG', 'ent': 'Oxford United'},
#           {'start': 45, 'end': 49, 'label': 2, 'type': 'PERSON', 'ent': 'Luke'},
#           {'start': 50, 'end': 55, 'label': 0, 'type': 'PERSON', 'ent': 'Kenny'},
#           {'start': 70, 'end': 91, 'label': 1, 'type': 'DATE', 'ent': 'the end of the season'}],
#  'hallucinations': ['Everton', 'Oxford United', 'midfielder', 'Luke', 'on loan', 'until the end of the season', 'with the option of a permanent move']
# },
{'id': 8770,
 'pred': 'Oxford United midfielder Callum Kenny has signed a new one-year contract with the League One club, keeping him at the club until the summer of 2018.',
 'ents': [{'start': 0, 'end': 13, 'label': 1, 'type': 'ORG', 'ent': 'Oxford United'},
          {'start': 25, 'end': 31, 'label': 2, 'type': 'PERSON', 'ent': 'Callum'},
          {'start': 32, 'end': 37, 'label': 0, 'type': 'PERSON', 'ent': 'Kenny'},
          {'start': 55, 'end': 63, 'label': 2, 'type': 'DATE', 'ent': 'one-year'},
          {'start': 82, 'end': 92, 'label': 2, 'type': 'ORG', 'ent': 'League One'},
          {'start': 129, 'end': 147, 'label': 2, 'type': 'DATE', 'ent': 'the summer of 2018'}],
 'hallucinations': ['Callum', 'one-year contract', 'the League One club', 'the summer of 2018']},
{'id': 5620,
 'pred': 'A second sinkhole has opened up in Manchester city centre, the latest in a series of sinkholes to open up in the city.',
 'ents': [{'start': 2, 'end': 8, 'label': 3, 'type': 'ORDINAL', 'ent': 'second'},
          {'start': 35, 'end': 45, 'label': 0, 'type': 'GPE', 'ent': 'Manchester'}],
 'hallucinations': ['second']
},
{'id': 1948,
 'pred': 'A mother and her partner have appeared in court charged in connection with the death of a six-year-old boy who was found dead at a water park.',
 'ents': [{'start': 90, 'end': 102, 'label': 2, 'type': 'DATE', 'ent': 'six-year-old'}],
 'hallucinations': ['six-year-old']
},
{'id': 10,
 'pred': 'A hospital where a patient died and another was left sitting on a bedpan for more than an hour in the emergency department has been ordered to improve its A&E services after inspectors found poor standards of care.',
 'ents': [{'start': 77, 'end': 94, 'label': 0, 'type': 'TIME', 'ent': 'more than an hour'},
          {'start': 155, 'end': 158, 'label': 0, 'type': 'ORG', 'ent': 'A&E'}],
 'hallucinations': []
},
{'id': 2156,
 'pred': "The public has been asked to vote for their favourite bird to be named as the UK's national bird ahead of the general election.",
 'ents': [{'start': 78, 'end': 80, 'label': 0, 'type': 'GPE', 'ent': 'UK'}],
 'hallucinations': []
},
{'id': 2991,
 'pred': "A man who hid his girlfriend's body in a cupboard and took her benefits for several months after her death has been jailed.",
 'ents': [{'start': 76, 'end': 90, 'label': 0, 'type': 'DATE', 'ent': 'several months'}],
 'hallucinations': []
},
{'id': 8702,
 'pred': 'Eritrea has accused the United Nations of carrying out an "unprecedented attack" on its government after it published a report accusing the Eritrean government of human rights abuses.',
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'GPE', 'ent': 'Eritrea'},
          {'start': 20, 'end': 38, 'label': 0, 'type': 'ORG', 'ent': 'the United Nations'},
          {'start': 140, 'end': 148, 'label': 0, 'type': 'NORP', 'ent': 'Eritrean'}],
 'hallucinations': ['"unprecedented attack"']
},
{'id': 3466,
 'pred': 'The body of a man has been found off the coast of St Fergus in Aberdeenshire, police have confirmed.',
 'ents': [{'start': 50, 'end': 59, 'label': 0, 'type': 'GPE', 'ent': 'St Fergus'},
          {'start': 63, 'end': 76, 'label': 1, 'type': 'GPE', 'ent': 'Aberdeenshire'}],
 'hallucinations': ['in Aberdeenshire']
},
{'id': 3088,
 'pred': "Adele's new single, Hello, has become the fastest-selling single of the year in the UK, with more than 200,000 downloads in its first 24 hours.",
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'PERSON', 'ent': 'Adele'},
          {'start': 84, 'end': 86, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 93, 'end': 110, 'label': 1, 'type': 'CARDINAL', 'ent': 'more than 200,000'},
          {'start': 134, 'end': 142, 'label': 2, 'type': 'TIME', 'ent': '24 hours'}],
 'hallucinations': []
},
{'id': 3088,
 'pred': "Adele's new single, Hello, has become the fastest-selling single of the year in the UK, with more than 200,000 downloads in its first week of release.",
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'PERSON', 'ent': 'Adele'},
          {'start': 84, 'end': 86, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 93, 'end': 110, 'label': 1, 'type': 'CARDINAL', 'ent': 'more than 200,000'},
          {'start': 124, 'end': 138, 'label': 1, 'type': 'DATE', 'ent': 'its first week'}],
 'hallucinations': ['the fastest-selling single of the year', 'more than 200,000 downloads in its first week of release']
},
{'id': 5706,
 'pred': "Kenya's main opposition leader, Raila Odinga, has called for a one-day strike on Monday to protest against the result of last week's presidential election.",
 'ents': [{'start': 0, 'end': 5, 'label': 0, 'type': 'GPE', 'ent': 'Kenya'},
          {'start': 32, 'end': 37, 'label': 1, 'type': 'PERSON', 'ent': 'Raila'},
          {'start': 38, 'end': 44, 'label': 0, 'type': 'PERSON', 'ent': 'Odinga'},
          {'start': 63, 'end': 70, 'label': 2, 'type': 'DATE', 'ent': 'one-day'},
          {'start': 81, 'end': 87, 'label': 2, 'type': 'DATE', 'ent': 'Monday'},
          {'start': 121, 'end': 132, 'label': 0, 'type': 'DATE', 'ent': "last week's"}],
 'hallucinations': ['Raila', 'one-day strike on Monday']
},
{'id': 5290,
 'pred': 'A hacking group based in North Korea is believed to have been behind the global ransomware attack that hit the NHS in May, the BBC has learned.',
 'ents': [{'start': 25, 'end': 36, 'label': 0, 'type': 'GPE', 'ent': 'North Korea'},
          {'start': 111, 'end': 114, 'label': 0, 'type': 'ORG', 'ent': 'NHS'},
          {'start': 118, 'end': 121, 'label': 0, 'type': 'DATE', 'ent': 'May'},
          {'start': 127, 'end': 130, 'label': 0, 'type': 'ORG', 'ent': 'BBC'}],
 'hallucinations': []
},
# {'id': 10306,
#  'pred': 'Taliban leader Mullah Akhtar Mansour and his bodyguards have been shot dead in the Pakistani city of Quetta, officials say, in the first such incident since his appointment last month.',
#  'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'ORG', 'ent': 'Taliban'},
#           {'start': 15, 'end': 21, 'label': 0, 'type': 'PERSON', 'ent': 'Mullah'},
#           {'start': 22, 'end': 28, 'label': 1, 'type': 'PERSON', 'ent': 'Akhtar'},
#           {'start': 29, 'end': 36, 'label': 0, 'type': 'PERSON', 'ent': 'Mansour'},
#           {'start': 83, 'end': 92, 'label': 0, 'type': 'NORP', 'ent': 'Pakistani'},
#           {'start': 101, 'end': 107, 'label': 0, 'type': 'GPE', 'ent': 'Quetta'},
#           {'start': 131, 'end': 136, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
#           {'start': 173, 'end': 183, 'label': -1, 'type': 'DATE', 'ent': 'last month'}],
#  'hallucinations': ['have been shot dead', 'officials say', 'Akhtar']
# },
{'id': 7871,
 'pred': "A man in his 20s has been arrested on suspicion of attempted murder after a man was stabbed outside a McDonald's in Londonderry.",
 'ents': [{'start': 13, 'end': 16, 'label': 0, 'type': 'DATE', 'ent': '20s'},
          {'start': 102, 'end': 110, 'label': 1, 'type': 'ORG', 'ent': 'McDonald'},
          {'start': 116, 'end': 127, 'label': 2, 'type': 'GPE', 'ent': 'Londonderry'}],
 'hallucinations': ['has been arrested', "outside a McDonald's in Londonderry."]
},
{'id': 6041,
 'pred': 'The owner of Merlin Entertainments is to appear in court over a rollercoaster crash at Alton Towers in which five people were injured.',
 'ents': [{'start': 13, 'end': 34, 'label': 0, 'type': 'ORG', 'ent': 'Merlin Entertainments'},
          {'start': 87, 'end': 99, 'label': 1, 'type': 'ORG', 'ent': 'Alton Towers'},
          {'start': 109, 'end': 113, 'label': 0, 'type': 'CARDINAL', 'ent': 'five'}],
 'hallucinations': ['The owner of Merlin Entertainments is to appear in court', 'at Alton Towers']
},
{'id': 1110,
 'pred': 'Chelsea manager Jose Mourinho said the Premier League is "not the best league in the world" after his side\'s 1-0 defeat by West Ham was like "football from the 19th Century".',
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'ORG', 'ent': 'Chelsea'},
          {'start': 16, 'end': 20, 'label': 0, 'type': 'PERSON', 'ent': 'Jose'},
          {'start': 21, 'end': 29, 'label': 0, 'type': 'PERSON', 'ent': 'Mourinho'},
          {'start': 35, 'end': 53, 'label': 0, 'type': 'ORG', 'ent': 'the Premier League'},
          {'start': 109, 'end': 112, 'label': 2, 'type': 'CARDINAL', 'ent': '1-0'},
          {'start': 123, 'end': 131, 'label': 0, 'type': 'LOC', 'ent': 'West Ham'},
          {'start': 156, 'end': 172, 'label': 0, 'type': 'DATE', 'ent': 'the 19th Century'}],
 'hallucinations': ['1-0 defeat by West Ham']
},
{'id': 8242,
 'pred': "Nigeria's main opposition party has lost control of all but one of the country's 36 states in the first elections since the end of military rule in 1999.",
 'ents': [{'start': 0, 'end': 7, 'label': 0, 'type': 'GPE', 'ent': 'Nigeria'},
          {'start': 81, 'end': 83, 'label': 1, 'type': 'CARDINAL', 'ent': '36'},
          {'start': 98, 'end': 103, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 148, 'end': 152, 'label': 0, 'type': 'DATE', 'ent': '1999'}],
 'hallucinations': ["lost control of all but one of the country's 36 states", 'in the first elections']
},
{'id': 9526,
 'pred': "The first World Cup was held in India in 1950, and the country's first football team, the Indian Football Federation (IFF), won a place in the final of the inaugural tournament.",
 'ents': [{'start': 4, 'end': 9, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 10, 'end': 19, 'label': 0, 'type': 'EVENT', 'ent': 'World Cup'},
          {'start': 32, 'end': 37, 'label': 0, 'type': 'GPE', 'ent': 'India'},
          {'start': 41, 'end': 45, 'label': 1, 'type': 'DATE', 'ent': '1950'},
          {'start': 65, 'end': 70, 'label': 0, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 86, 'end': 116, 'label': 2, 'type': 'ORG', 'ent': 'the Indian Football Federation'}],
 'hallucinations': ['The first World Cup was held in India in 1950', 'Indian Football Federation (IFF)']
},
{'id': 9340,
 'pred': 'Cardiff Blues scrum-half Mike Phillips has been suspended from the Wales squad after being involved in an incident outside a restaurant in Cardiff.',
 'ents': [{'start': 0, 'end': 13, 'label': 0, 'type': 'ORG', 'ent': 'Cardiff Blues'},
          {'start': 25, 'end': 29, 'label': 0, 'type': 'PERSON', 'ent': 'Mike'},
          {'start': 30, 'end': 38, 'label': 0, 'type': 'PERSON', 'ent': 'Phillips'},
          {'start': 67, 'end': 72, 'label': 0, 'type': 'GPE', 'ent': 'Wales'},
          {'start': 139, 'end': 146, 'label': 0, 'type': 'GPE', 'ent': 'Cardiff'}],
 'hallucinations': []},
{'id': 11046,
 'pred': 'A man who led police on a high-speed chase for more than 50 minutes after he was caught driving without a licence has been jailed.',
 'ents': [{'start': 47, 'end': 67, 'label': 0, 'type': 'TIME', 'ent': 'more than 50 minutes'}],
 'hallucinations': []},
{'id': 4947, # NOTE
 'pred': 'Universities in England and Wales are to stage a one-day strike on Wednesday in a dispute over pay, in a row over the size of a pay rise.',
 'ents': [{'start': 16, 'end': 23, 'label': 1, 'type': 'GPE', 'ent': 'England'},
          {'start': 28, 'end': 33, 'label': 1, 'type': 'GPE', 'ent': 'Wales'},
          {'start': 49, 'end': 56, 'label': 2, 'type': 'DATE', 'ent': 'one-day'},
          {'start': 67, 'end': 76, 'label': 3, 'type': 'DATE', 'ent': 'Wednesday'}],
 'hallucinations': []},
{'id': 3858,
 'pred': 'Gardeners in Northern Ireland are being urged to get rid of a "thug" plant that is out-competing bluebells in their gardens.',
 'ents': [{'start': 13, 'end': 29, 'label': 2, 'type': 'GPE', 'ent': 'Northern Ireland'}],
 'hallucinations': ['in Northern Ireland']},
{'id': 1843, # how is this possbile
 'pred': 'A five-year-old boy with a rare form of cancer who was cheered on by Sunderland fans when he appeared as a mascot for the team has lost his fight against the disease.',
 'ents': [{'start': 2, 'end': 15, 'label': 1, 'type': 'DATE', 'ent': 'five-year-old'},
          {'start': 69, 'end': 79, 'label': 0, 'type': 'ORG', 'ent': 'Sunderland'}],
 'hallucinations': []},
{'id': 4867,
 'pred': 'Work permits for EU citizens living and working in the UK could cut net migration by up to 100,000 a year if the UK voted to leave the EU, according to a report from a think tank.',
 'ents': [{'start': 17, 'end': 19, 'label': 0, 'type': 'ORG', 'ent': 'EU'},
          {'start': 55, 'end': 57, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 85, 'end': 98, 'label': 0, 'type': 'CARDINAL', 'ent': 'up to 100,000'},
          {'start': 113, 'end': 115, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 135, 'end': 137, 'label': 0, 'type': 'ORG', 'ent': 'EU'}],
 'hallucinations': []},
{'id': 1472,
 'pred': 'A man armed with a knife has robbed a van driver in Edinburgh after he dropped off a man he had picked up in the city centre.',
 'ents': [{'start': 52, 'end': 61, 'label': 1, 'type': 'GPE', 'ent': 'Edinburgh'}],
 'hallucinations': []},
{'id': 10112,
 'pred': 'Three men and two teenagers have been arrested following disturbances at the Celtic v Aberdeen match at Pittodrie on Friday night.',
 'ents': [{'start': 0, 'end': 5, 'label': 3, 'type': 'CARDINAL', 'ent': 'Three'},
          {'start': 14, 'end': 17, 'label': 0, 'type': 'CARDINAL', 'ent': 'two'},
          {'start': 77, 'end': 83, 'label': 1, 'type': 'NORP', 'ent': 'Celtic'},
          {'start': 86, 'end': 94, 'label': 0, 'type': 'ORG', 'ent': 'Aberdeen'},
          {'start': 104, 'end': 113, 'label': 1, 'type': 'ORG', 'ent': 'Pittodrie'},
          {'start': 117, 'end': 123, 'label': 1, 'type': 'DATE', 'ent': 'Friday'}],
 'hallucinations': []},
{'id': 9203,
 'pred': 'More than 100 drivers a day have been caught speeding on a stretch of the M4 in Neath Port Talbot since speed cameras went into operation, figures have shown.',
 'ents': [{'start': 80, 'end': 97, 'label': 1, 'type': 'ORG', 'ent': 'Neath Port Talbot'}],
 'hallucinations': []},
{'id': 9166,
 'pred': 'Port Vale came from behind to beat West Bromwich Albion and reach the second round of the League One play-off semi-finals.',
 'ents': [{'start': 0, 'end': 9, 'label': 1, 'type': 'ORG', 'ent': 'Port Vale'},
          {'start': 35, 'end': 55, 'label': 1, 'type': 'ORG', 'ent': 'West Bromwich Albion'},
          {'start': 70, 'end': 76, 'label': 2, 'type': 'ORDINAL', 'ent': 'second'}],
 'hallucinations': ["the second round of the League One play-off semi-finals."]},
{'id': 6066,
 'pred': "England head coach Eddie Jones has defended the targeting of Ireland fly-half Johnny Sexton in Saturday's Six Nations match against England, saying it is part of the game.",
 'ents': [{'start': 0, 'end': 7, 'label': None, 'type': 'GPE', 'ent': 'England'},
          {'start': 19, 'end': 24, 'label': None, 'type': 'PERSON', 'ent': 'Eddie'},
          {'start': 25, 'end': 30, 'label': None, 'type': 'PERSON', 'ent': 'Jones'},
          {'start': 61, 'end': 68, 'label': None, 'type': 'GPE', 'ent': 'Ireland'},
          {'start': 78, 'end': 84, 'label': None, 'type': 'PERSON', 'ent': 'Johnny'},
          {'start': 85, 'end': 91, 'label': None, 'type': 'PERSON', 'ent': 'Sexton'},
          {'start': 95, 'end': 103, 'label': None, 'type': 'DATE', 'ent': 'Saturday'},
          {'start': 106, 'end': 117, 'label': None, 'type': 'ORG', 'ent': 'Six Nations'},
          {'start': 132, 'end': 139, 'label': None, 'type': 'GPE', 'ent': 'England'}],
 'hallucinations': []},
{'id': 2333,
 'pred': "Donald Trump's decision to pull the US out of the Paris Agreement on climate change was a blow to Rex Tillerson.",
 'ents': [{'start': 0, 'end': 6, 'label': 1, 'type': 'PERSON', 'ent': 'Donald'},
          {'start': 7, 'end': 12, 'label': 0, 'type': 'PERSON', 'ent': "Trump"},
          {'start': 36, 'end': 38, 'label': 0, 'type': 'GPE', 'ent': 'US'},
          {'start': 46, 'end': 65, 'label': 0, 'type': 'ORG', 'ent': 'the Paris Agreement'},
          {'start': 98, 'end': 101, 'label': 1, 'type': 'PERSON', 'ent': 'Rex'},
          {'start': 102, 'end': 111, 'label': 0, 'type': 'PERSON', 'ent': 'Tillerson'}],
 'hallucinations': []},
{'id': 10898,
 'pred': 'Amnesty International has called on the Indian government to repeal a controversial law that gives security forces sweeping powers in Indian-administered Kashmir, saying it has contributed to human rights abuses.',
 'ents': [{'start': 0, 'end': 21, 'label': 0, 'type': 'ORG', 'ent': 'Amnesty International'},
          {'start': 40, 'end': 46, 'label': 0, 'type': 'NORP', 'ent': 'Indian'},
          {'start': 134, 'end': 140, 'label': 0, 'type': 'NORP', 'ent': 'Indian'},
          {'start': 154, 'end': 161, 'label': 0, 'type': 'LOC', 'ent': 'Kashmir'}],
 'hallucinations': []},
{'id': 8921,
 'pred': 'The UK\'s Department for Environment, Food and Rural Affairs (Defra) has called for "more realistic field trials" of two common pesticides after research suggested they could be damaging to honey bees.',
 'ents': [{'start': 4, 'end': 6, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
          {'start': 9, 'end': 59, 'label': 1, 'type': 'ORG', 'ent': 'Department for Environment, Food and Rural Affairs'},
          {'start': 116, 'end': 119, 'label': 0, 'type': 'CARDINAL', 'ent': 'two'}],
 'hallucinations': []},
{'id': 6427,
 'pred': 'Crystal Palace earned their first Premier League win of the season as they came from behind to beat Everton at Goodison Park for the second time this season.',
 'ents': [{'start': 0, 'end': 14, 'label': 0, 'type': 'ORG', 'ent': 'Crystal Palace'},
          {'start': 28, 'end': 33, 'label': None, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 34, 'end': 48, 'label': None, 'type': 'ORG', 'ent': 'Premier League'},
          {'start': 100, 'end': 107, 'label': 1, 'type': 'GPE', 'ent': 'Everton'},
          {'start': 111, 'end': 124, 'label': 1, 'type': 'FAC', 'ent': 'Goodison Park'},
          {'start': 133, 'end': 139, 'label': None, 'type': 'ORDINAL', 'ent': 'second'},
          {'start': 145, 'end': 156, 'label': None, 'type': 'DATE', 'ent': 'this season'}],
 'hallucinations': []},
{'id': 572,
 'pred': 'A police investigation has been launched into the disappearance of a car that has been missing from a Hertfordshire garage for almost 20 years.',
 'ents': [{'start': 134, 'end': 142, 'label': 0, 'type': 'DATE', 'ent': '20 years'}],
 'hallucinations': []},
{'id': 5113,
 'pred': 'A volcano in Costa Rica has erupted for the first time in six years, spewing huge plumes of ash and gas into the sky over the capital and surrounding areas.',
 'ents': [{'start': 13, 'end': 23, 'label': 0, 'type': 'GPE', 'ent': 'Costa Rica'},
          {'start': 44, 'end': 49, 'label': 2, 'type': 'ORDINAL', 'ent': 'first'},
          {'start': 58, 'end': 67, 'label': 0, 'type': 'DATE', 'ent': 'six years'}],
 'hallucinations': ['for the first time']},
{'id': 6920,
 'pred': 'A Dundee United fan has admitted striking a Rangers fan in the head with a metal pole at the end of the Scottish Premiership match between the sides.',
 'ents': [{'start': 2, 'end': 19, 'label': 0, 'type': 'ORG', 'ent': 'Dundee United fan'},
          {'start': 44, 'end': 51, 'label': 0, 'type': 'ORG', 'ent': 'Rangers'},
          {'start': 104, 'end': 112, 'label': 2, 'type': 'NORP', 'ent': 'Scottish'}],
 'hallucinations': []},
{'id': 3590,
 'pred': 'A court in New York has ordered the trial of Hank Greenberg, the former co-founder of US insurance giant AIG, to begin next month on charges of fraud.',
 'ents': [{'start': 11,
   'end': 19,
   'label': 0,
   'type': 'GPE',
   'ent': 'New York'},
  {'start': 45, 'end': 49, 'label': 0, 'type': 'PERSON', 'ent': 'Hank'},
  {'start': 50,
   'end': 59,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Greenberg'},
  {'start': 86, 'end': 88, 'label': 0, 'type': 'GPE', 'ent': 'US'},
  {'start': 105, 'end': 108, 'label': 1, 'type': 'PRODUCT', 'ent': 'AIG'},
  {'start': 119,
   'end': 129,
   'label': 2,
   'type': 'DATE',
   'ent': 'next month'}],
 'hallucinations': []},
{'id': 3278,
 'pred': "French centrist presidential candidate Emmanuel Macron has accepted an offer to form an alliance with a former prime minister to fight far-right candidate Marine Le Pen in next month's election.",
 'ents': [{'start': 0, 'end': 6, 'label': 0, 'type': 'NORP', 'ent': 'French'},
          {'start': 39, 'end': 47, 'label': 1, 'type': 'PERSON', 'ent': 'Emmanuel'},
          {'start': 48, 'end': 54, 'label': 0, 'type': 'PERSON', 'ent': 'Macron'},
          {'start': 155, 'end': 161, 'label': 0, 'type': 'PERSON', 'ent': 'Marine'},
          {'start': 162, 'end': 164, 'label': 0, 'type': 'PERSON', 'ent': 'Le'},
          {'start': 165, 'end': 168, 'label': 0, 'type': 'PERSON', 'ent': 'Pen'},
          {'start': 172, 'end': 184, 'label': 2, 'type': 'DATE', 'ent': "next month's"}],
 'hallucinations': []},
{'id': 6117,
 'pred': 'A Porto-based football player has been banned for four years for attacking a referee and three months for assaulting an opposition player in an amateur league match in the Portuguese city.',
 'ents': [{'start': 2,
   'end': 7,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Porto'},
  {'start': 50, 'end': 60, 'label': 0, 'type': 'DATE', 'ent': 'four years'},
  {'start': 89,
   'end': 101,
   'label': 0,
   'type': 'DATE',
   'ent': 'three months'},
  {'start': 172,
   'end': 182,
   'label': 1,
   'type': 'NORP',
   'ent': 'Portuguese'}],
 'hallucinations': []},
{'id': 7688,
 'pred': 'Sam Baldock scored the only goal of the game as Brighton beat Wolves to move up to third in the Championship with victory at the Amex Stadium.',
 'ents': [{'start': 0,
   'end': 3,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Sam'},
  {'start': 4, 'end': 11, 'label': 0, 'type': 'PERSON', 'ent': 'Baldock'},
  {'start': 83, 'end': 88, 'label': 2, 'type': 'ORDINAL', 'ent': 'third'},
  {'start': 125,
   'end': 141,
   'label': 2,
   'type': 'FAC',
   'ent': 'the Amex Stadium'}],
 'hallucinations': []},
{'id': 9758,
 'pred': 'Stoke City defender Ryan Shawcross has been ruled out for up to four weeks with a hamstring injury and could miss the rest of the season.',
 'ents': [{'start': 20,
   'end': 24,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Ryan'},
  {'start': 25,
   'end': 34,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Shawcross'},
  {'start': 58,
   'end': 74,
   'label': 0,
   'type': 'DATE',
   'ent': 'up to four weeks'}],
 'hallucinations': []},
{'id': 1733,
 'pred': "A man has been found guilty of the manslaughter of his mother's partner who died after a fight at a Dorset holiday park.",
 'ents': [{'start': 100,
   'end': 106,
   'label': 2,
   'type': 'GPE',
   'ent': 'Dorset'}],
 'hallucinations': []},
{'id': 3432,
 'pred': 'A search is under way to find a mystery photographer who took the last photograph of a woman before she died, after she was found dead in a garden at a Kent castle.',
 'ents': [{'start': 152,
   'end': 156,
   'label': 3,
   'type': 'PERSON',
   'ent': 'Kent'}],
 'hallucinations': []},
{'id': 1264,
 'pred': 'Former Scotland boss Alex McLeish has agreed a deal to become the new coach of Egyptian club Zamalek, the club have confirmed.',
 'ents': [{'start': 7,
   'end': 15,
   'label': 0,
   'type': 'GPE',
   'ent': 'Scotland'},
  {'start': 21, 'end': 25, 'label': 0, 'type': 'PERSON', 'ent': 'Alex'},
  {'start': 26, 'end': 33, 'label': 0, 'type': 'PERSON', 'ent': 'McLeish'},
  {'start': 79, 'end': 87, 'label': 0, 'type': 'NORP', 'ent': 'Egyptian'},
  {'start': 93,
   'end': 100,
   'label': 0,
   'type': 'PERSON',
   'ent': 'Zamalek'}],
 'hallucinations': []},
{'id': 5195,
 'pred': 'A Church of Ireland bishop has said the church is "not up to pace" with the debate on same-sex marriage in Northern Ireland.',
 'ents': [{'start': 2,
   'end': 19,
   'label': 0,
   'type': 'ORG',
   'ent': 'Church of Ireland'},
  {'start': 107,
   'end': 123,
   'label': 0,
   'type': 'GPE',
   'ent': 'Northern Ireland'}],
 'hallucinations': []},
{'id': 1436,
 'pred': 'Labour leader Jeremy Corbyn has said he would allow his MPs a "free vote" on whether the UK should take part in air strikes against Islamic State militants in Syria.',
 'ents': [{'start': 14,
   'end': 20,
   'label': 1,
   'type': 'PERSON',
   'ent': 'Jeremy'},
  {'start': 21, 'end': 27, 'label': 0, 'type': 'PERSON', 'ent': 'Corbyn'},
  {'start': 89, 'end': 91, 'label': 0, 'type': 'GPE', 'ent': 'UK'},
  {'start': 132, 'end': 139, 'label': 0, 'type': 'NORP', 'ent': 'Islamic'},
  {'start': 159, 'end': 164, 'label': 0, 'type': 'GPE', 'ent': 'Syria'}],
 'hallucinations': []}
]

In [11]:
len(annotated)

120

In [12]:
ID = randint(0, len(xsum_preds))
print(ID)

11169


In [13]:
new_dict = {}
new_dict['id'] = ID
new_dict['pred'] = xsum_preds[ID]
new_dict['ents'] = []

for ent in nlp(new_dict['pred']).to_json()['ents']:
    if ent['label'] != 'PERSON':
        new_dict['ents'].append(ent)
    else:
        name_parts = new_dict['pred'][ent['start']: ent['end']].split()
        init_start = ent['start']
        for p in name_parts:
            new_dict['ents'].append({'start': init_start, 'end': init_start + len(p), 'label': 'PERSON'})
            init_start = init_start + len(p) + 1
            
for e in new_dict['ents']:
    e['type'] = e['label']
    e['ent'] = new_dict['pred'][e['start']: e['end']]
    e['label'] = None

new_dict['hallucinations'] = []

In [14]:
new_dict

{'id': 11169,
 'pred': 'Controversial plans for a new golf course in Sutherland have been submitted to the Scottish government for approval.',
 'ents': [{'start': 45,
   'end': 55,
   'label': None,
   'type': 'ORG',
   'ent': 'Sutherland'},
  {'start': 83, 'end': 91, 'label': None, 'type': 'NORP', 'ent': 'Scottish'}],
 'hallucinations': []}

In [15]:
for i, e in enumerate(new_dict['ents']):
    print('{} - {}'.format(i, new_dict['pred'][e['start']: e['end']]))

print('- target:')
print(xsum_target[ID])
print('\n- prediction:')
print(xsum_preds[ID])
print('\n- source:')
print(xsum_source[ID])

0 - Sutherland
1 - Scottish
- target:
The developers of a planned 18-hole championship golf course have sought to allay concerns about their proposal's impact on a protected coastal area.

- prediction:
Controversial plans for a new golf course in Sutherland have been submitted to the Scottish government for approval.

- source:
US-based golf course designers Mike Keiser, Bill Coore and Todd Warnock have proposed creating the course at Coul Links, near Dornoch. Four conservation charities have joined forces to oppose the project. The developers said they were aiming for "minimal intrusion on the landscape". The Scottish Wildlife Trust, RSPB Scotland, Buglife and Plantlife have concerns the course will result in the loss of a protected sand dunes habitat and rare wildlife. The developers, who have held two meetings in the local area as part of public consultation on the plans, said the building of the course would involve laying turf over less than 55 acres (22ha) within a Site of Speci

In [16]:
import json

with open('annotated.json', 'w') as fout:
    json.dump(annotated , fout)

#### Check Data Leaking

In [17]:
train_source_path =  PATH['xsum_fariseq'] + '/train.source'

In [18]:
# xsum_train_source = read_lines(train_source_path)

In [19]:
# for i, s in enumerate(xsum_train_source):
#     if 'Bradley Lowery' in s and 'Sunderland' in s and 'five' in s:
#         # print('{}: {}'.format(i, s))
#         print(i)

In [20]:
# xsum_train_source[35382]

In [21]:
len('a referee and ')

14